In [53]:
import geopandas as gpd

world_path = r"C:/Users/paulk/My_Drive/Statistik/Statistik_Projekt/additional_data/custom.geo.json"

def load_world_geojson(path):
    world = gpd.read_file(path)

    if "name" in world.columns:
        world = world.rename(columns={"name": "country"})
    elif "admin" in world.columns:
        world = world.rename(columns={"admin": "country"})
    elif "ADMIN" in world.columns:
        world = world.rename(columns={"ADMIN": "country"})
    else:
        raise ValueError("Kein gültiger Länderschlüssel gefunden!")

    world["lon"] = world.geometry.centroid.x
    world["lat"] = world.geometry.centroid.y

    return world

world = load_world_geojson(world_path)
GEOJSON_COUNTRIES = set(world["country"])

print("Anzahl Länder:", len(GEOJSON_COUNTRIES))
print(sorted(list(GEOJSON_COUNTRIES))[:20])


Anzahl Länder: 176
['Afghanistan', 'Albania', 'Algeria', 'Angola', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herz.', 'Botswana', 'Brazil']


C:\Users\paulk\AppData\Local\Temp\ipykernel_90496\3253173303.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  world["lon"] = world.geometry.centroid.x
C:\Users\paulk\AppData\Local\Temp\ipykernel_90496\3253173303.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  world["lat"] = world.geometry.centroid.y


In [54]:
# %%
import pandas as pd

csv_path = r"C:/Users/paulk/My_Drive/Statistik/Statistik_Projekt/archive/time_series.csv"

def load_unhcr_data(path):
    return pd.read_csv(path)

df = load_unhcr_data(csv_path)

print(df.head())
print("\nSpalten:", df.columns.tolist())
print("Anzahl Zeilen:", len(df))


   Year Country / territory of asylum/residence           Origin  \
0  1951                               Australia  Various/Unknown   
1  1951                                 Austria  Various/Unknown   
2  1951                                 Belgium  Various/Unknown   
3  1951                                  Canada  Various/Unknown   
4  1951                             Switzerland  Various/Unknown   

                            Population type   Value  
0  Refugees (incl. refugee-like situations)  180000  
1  Refugees (incl. refugee-like situations)  282000  
2  Refugees (incl. refugee-like situations)   55000  
3  Refugees (incl. refugee-like situations)  168511  
4  Refugees (incl. refugee-like situations)   10000  

Spalten: ['Year', 'Country / territory of asylum/residence', 'Origin', 'Population type', 'Value']
Anzahl Zeilen: 298441


C:\Users\paulk\AppData\Local\Temp\ipykernel_90496\2066054342.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


In [55]:
# %%
unhcr_origins = set(df["Origin"].dropna().unique())
unhcr_asylums = set(df["Country / territory of asylum/residence"].dropna().unique())

all_unhcr_names = unhcr_origins.union(unhcr_asylums)

print("UNHCR Länder insgesamt:", len(all_unhcr_names))
print(sorted(list(all_unhcr_names))[:50])


UNHCR Länder insgesamt: 226
['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Rep.', 'Chad', 'Chile', 'China', 'China, Hong Kong SAR', 'China, Macao SAR', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica']


In [56]:
# %%
exact_matches = sorted([c for c in all_unhcr_names if c in GEOJSON_COUNTRIES])
missing = sorted([c for c in all_unhcr_names if c not in GEOJSON_COUNTRIES])

print("Exakte Matches:", len(exact_matches))
print("Fehlende Länder:", len(missing))
print("\nBeispiele fehlende Länder:")
print(missing[:30])


Exakte Matches: 146
Fehlende Länder: 80

Beispiele fehlende Länder:
['American Samoa', 'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Bahrain', 'Barbados', 'Bermuda', 'Bolivia (Plurinational State of)', 'Bonaire', 'Bosnia and Herzegovina', 'British Virgin Islands', 'Brunei Darussalam', 'Cabo Verde', 'Cayman Islands', 'China, Hong Kong SAR', 'China, Macao SAR', 'Comoros', 'Cook Islands', 'Curaçao', 'Czech Rep.', "Dem. People's Rep. of Korea", 'Dem. Rep. of the Congo', 'Dominica', 'Equatorial Guinea', 'French Guiana', 'French Polynesia', 'Gibraltar', 'Grenada', 'Guadeloupe']


In [57]:
# %%
import difflib

def fuzzy_match(name, choices, cutoff=0.75):
    return difflib.get_close_matches(name, choices, n=3, cutoff=cutoff)

for c in missing[:30]:
    print(f"{c:35} → {fuzzy_match(c, GEOJSON_COUNTRIES)}")


American Samoa                      → []
Andorra                             → []
Anguilla                            → []
Antigua and Barbuda                 → []
Aruba                               → []
Bahrain                             → []
Barbados                            → []
Bermuda                             → []
Bolivia (Plurinational State of)    → []
Bonaire                             → []
Bosnia and Herzegovina              → ['Bosnia and Herz.']
British Virgin Islands              → []
Brunei Darussalam                   → []
Cabo Verde                          → []
Cayman Islands                      → []
China, Hong Kong SAR                → []
China, Macao SAR                    → []
Comoros                             → []
Cook Islands                        → []
Curaçao                             → []
Czech Rep.                          → []
Dem. People's Rep. of Korea         → []
Dem. Rep. of the Congo              → ['Dem. Rep. Congo']
Dominica              

In [58]:
COUNTRY_REMAP = {

    # --- Länder mit abweichender Schreibweise ---
    "Bolivia (Plurinational State of)": "Bolivia",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "Cabo Verde": "Cape Verde",
    "Czech Rep.": "Czechia",
    "Côte d'Ivoire": "Ivory Coast",
    "Dem. People's Rep. of Korea": "North Korea",
    "Dem. Rep. of the Congo": "Dem. Rep. Congo",
    "Dominican Rep.": "Dominican Republic",
    "Iran (Islamic Rep. of)": "Iran",
    "Lao People's Dem. Rep.": "Laos",
    "Rep. of Korea": "South Korea",
    "Rep. of Moldova": "Moldova",
    "Russian Federation": "Russia",
    "Syrian Arab Rep.": "Syria",
    "United Rep. of Tanzania": "United Republic of Tanzania",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Viet Nam": "Vietnam",
    "Swaziland": "eSwatini",
    "The former Yugoslav Republic of Macedonia": "North Macedonia",

    # --- Politische Spezialfälle ---
    "Serbia and Kosovo (S/RES/1244 (1999))": "Republic of Serbia",
    "State of Palestine": "Palestine",
    "Palestinian": "Palestine",
    "Holy See (the)": "Vatican",

    # --- Territorien, die Natural Earth zusammenfasst ---
    "French Guiana": "France",
    "Guadeloupe": "France",
    "Martinique": "France",
    "Saint-Pierre-et-Miquelon": "France",
    "Puerto Rico": "United States of America",

    "Aruba": "Netherlands",
    "Curaçao": "Netherlands",
    "Bonaire": "Netherlands",
    "Sint Maarten (Dutch part)": "Netherlands",

    "Gibraltar": "United Kingdom",
    "Svalbard and Jan Mayen": "Norway",

    # --- Sehr kleine Territorien → fehlen in GeoJSON → ignorieren ---
    "American Samoa": None,
    "Anguilla": None,
    "Antigua and Barbuda": None,
    "Bermuda": None,
    "British Virgin Islands": None,
    "Cook Islands": None,
    "Cayman Islands": None,
    "Comoros": None,
    "Equatorial Guinea": None,
    "French Polynesia": None,
    "Kiribati": None,
    "Marshall Islands": None,
    "Micronesia (Federated States of)": None,
    "Montserrat": None,
    "Nauru": None,
    "Niue": None,
    "Norfolk Island": None,
    "Palau": None,
    "Saint Kitts and Nevis": None,
    "Saint Lucia": None,
    "Saint Vincent and the Grenadines": None,
    "Samoa": None,
    "San Marino": None,
    "Seychelles": None,
    "Solomon Islands": None,
    "Tonga": None,
    "Turks and Caicos Islands": None,
    "Tuvalu": None,
    "Wallis and Futuna Islands ": None,

    # --- Nicht-Länder → komplett ignorieren ---
    "Stateless": None,
    "Various/Unknown": None,
    "Tibetan": None,
}


In [59]:
# %%
def normalize_name(name: str):
    name = name.strip()

    if name in COUNTRY_REMAP:
        return COUNTRY_REMAP[name]

    if name in GEOJSON_COUNTRIES:
        return name

    return None

normalized_test = {c: normalize_name(c) for c in all_unhcr_names}

print("Total:", len(normalized_test))
print("Mapping None:", sum(v is None for v in normalized_test.values()))


Total: 226
Mapping None: 49


In [60]:
# %%
def filter_year(df, year):
    df_year = df[df["Year"] == year].copy()

    df_year["Origin_clean"] = df_year["Origin"].apply(normalize_name)
    df_year["Asylum_clean"] = df_year["Country / territory of asylum/residence"].apply(normalize_name)

    df_year = df_year.dropna(subset=["Origin_clean", "Asylum_clean"])

    return df_year

YEAR = 2016
df_year = filter_year(df, YEAR)

print("Original rows:", len(df[df["Year"] == YEAR]))
print("After cleaning:", len(df_year))
df_year.head()


Original rows: 47754
After cleaning: 43750


,Year,Country / territory of asylum/residence,Origin,Population type,Value,Origin_clean,Asylum_clean
250687,2016,Aruba,Colombia,Asylum-seekers,*,Colombia,Netherlands
250688,2016,Aruba,Colombia,Internally displaced persons,0,Colombia,Netherlands
250689,2016,Aruba,Colombia,Others of concern,0,Colombia,Netherlands
250690,2016,Aruba,Colombia,Returned IDPs,0,Colombia,Netherlands
250691,2016,Aruba,Colombia,Refugees (incl. refugee-like situations),0,Colombia,Netherlands


In [61]:
# %%
def aggregate_flows(df_year):
    return (
        df_year
        .groupby(["Origin_clean", "Asylum_clean"])["Value"]
        .sum()
        .reset_index()
    )

flows = aggregate_flows(df_year)

print("Number of flows:", len(flows))
flows.head()


Number of flows: 6233


,Origin_clean,Asylum_clean,Value
0,Afghanistan,Afghanistan,017975511142210000
1,Afghanistan,Algeria,0000*00
2,Afghanistan,Angola,*000000
3,Afghanistan,Argentina,*000500
4,Afghanistan,Armenia,*000000


In [62]:
# %%
def attach_coordinates(flows, world):
    flows = flows.merge(
        world[["country", "lon", "lat"]],
        left_on="Origin_clean",
        right_on="country",
        how="left"
    ).rename(columns={"lon": "origin_lon", "lat": "origin_lat"})

    flows = flows.merge(
        world[["country", "lon", "lat"]],
        left_on="Asylum_clean",
        right_on="country",
        how="left"
    ).rename(columns={"lon": "asylum_lon", "lat": "asylum_lat"})

    flows = flows.drop(columns=["country_x", "country_y"])
    return flows

flows_with_coords = attach_coordinates(flows, world)

print("Rows with coords:", len(flows_with_coords))
flows_with_coords.head()


Rows with coords: 6233


,Origin_clean,Asylum_clean,Value,origin_lon,origin_lat,asylum_lon,asylum_lat
0,Afghanistan,Afghanistan,017975511142210000,66.08669,33.856399,66.086690,33.856399
1,Afghanistan,Algeria,0000*00,66.08669,33.856399,2.598048,28.185481
2,Afghanistan,Angola,*000000,66.08669,33.856399,17.470573,-12.245869
3,Afghanistan,Argentina,*000500,66.08669,33.856399,-65.175361,-35.446821
4,Afghanistan,Armenia,*000000,66.08669,33.856399,45.000290,40.216608


In [63]:
# %%
flows_with_coords.to_csv(
    "output_csv_files/flows_with_coords_2016.csv",
    index=False
)

world.to_file(
    "output_csv_files/world_with_centroids.geojson",
    driver="GeoJSON"
)

print("Flows und Weltkarte gespeichert.")


Flows und Weltkarte gespeichert.


In [64]:
# %%
# === FLOWS FÜR ALLE JAHRE SPEICHERN (für GIFs & Notebook 3) ===

START_YEAR = 1970
END_YEAR = df["Year"].max()

all_flows = []

# WICHTIG: Value einmal sauber numerisch machen
df["Value"] = pd.to_numeric(df["Value"], errors="coerce")

for year in range(START_YEAR, END_YEAR + 1):
    df_year = filter_year(df, year)

    if df_year.empty:
        continue

    flows = aggregate_flows(df_year)
    flows_coords = attach_coordinates(flows, world)

    flows_coords["Year"] = year
    all_flows.append(flows_coords)

flows_all_years = pd.concat(all_flows, ignore_index=True)

# EXAKTES FORMAT wie im Ursprungs-Code
flows_all_years.to_csv(
    "output_csv_files/flows_with_coords_all_years.csv",
    index=False
)

print("Gespeichert:", len(flows_all_years), "Flows")
print("Jahre:", flows_all_years["Year"].min(), "-", flows_all_years["Year"].max())


Gespeichert: 112797 Flows
Jahre: 1970 - 2016
